## **Using BART MNLI Model for Zero Shot text Classification**

In [1]:
from transformers import pipeline
import csv

In [23]:
#I wanna see if in txt file there is any ';' 
def check_semicolon(file):
    with open(file, 'r') as f:
        for line in f:
            if ';' in line:
                return True
    return False

#Let's apply the function to the txt file
file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/test_shuffle.txt'
check_semicolon(file_path)




False

In [24]:
# Load the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Defining the categories
categories = ["Politics", "Health", "Finance", "Travel", "Food", "Education", "Environment", "Fashion", "Science", "Sports", "Technology", "Entertainment"]

# Loading the sentences from the .txt file
file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/test_shuffle.txt'
sentences = open(file_path, 'r').readlines()

# Classifying each sentence and collecting the results
results = []
for sentence in sentences:
    classification = classifier(sentence.strip(), categories, multi_class=True)
    # Getting the highest scoring category
    top_category = classification['labels'][0]
    results.append([sentence.strip(), top_category])

# Saving the results to a CSV file
csv_file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/classification_results.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')
    writer.writerow(["ID", "Label"])  # Writing header
    writer.writerows(results)

print("Classification completed and saved to CSV.")


The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf

Classification completed and saved to CSV.


In [13]:
import pandas as pd
df = pd.read_csv('C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/classification_results_2.csv')
df.head()

,ID,Label
0,The role of credit scores in lending decisions...,Finance
1,The impact of overpopulation on the environmen...,Environment
2,The importance of the scientific method in con...,Science
3,The startup accelerator provides funding and m...,Technology
4,"The benefits of biomimicry are many, including...",Technology


In [14]:
df.shape

(1140, 2)

In [15]:
#Now what I wanna do is in the ID column replace each sentence with its index
df['ID'] = df.index
df.head()

,ID,Label
0,0,Finance
1,1,Environment
2,2,Science
3,3,Technology
4,4,Technology


In [17]:
#Let's save the new dataframe to a new csv file
df.to_csv('C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/classification_results_3.csv', index=False)


## **Hybrid approach**

In [4]:
import pandas as pd

In [3]:
# Initialize the zero-shot classification pipeline with BART MNLI
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define your categories
categories = ["Politics", "Health", "Finance", "Travel", "Food", "Education", "Environment", "Fashion", "Science", "Sports", "Technology", "Entertainment"]

# Loading the sentences from the .txt file
file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Data/test_shuffle.txt'
sentences = open(file_path, 'r').readlines()

# Define the probability threshold
probability_threshold = 0.7

# Store results
classified_sentences = []

for sentence in sentences:
    # Classify and obtain probabilities
    result = classifier(sentence, categories, multi_class=True)
    
    # Filter based on the threshold
    probs = result['scores']
    labels = result['labels']
    filtered_labels = [label for label, prob in zip(labels, probs) if prob > probability_threshold]
    
    # Check if exactly one class's probability exceeds the threshold
    if len(filtered_labels) == 1:
        classified_sentences.append({
            'sentence': sentence,
            'category': filtered_labels[0],
            'probability': probs[labels.index(filtered_labels[0])]
        })

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf

NameError: name 'pd' is not defined

In [5]:
# Convert to DataFrame for easy manipulation or saving
df_classified = pd.DataFrame(classified_sentences)
df_classified.to_csv('C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/classification_forDA.csv', index=False)
print("Classification completed and saved to CSV.")


Classification completed and saved to CSV.


Ok let's try and augment the training set using these confident classifications : 

In [4]:
import pandas as pd

df = pd.read_csv('C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/classification_forDA.csv')
df

,sentence,category,probability
0,The role of credit scores in lending decisions...,Finance,0.709732
1,The impact of overpopulation on the environmen...,Environment,0.774633
2,The importance of the scientific method in con...,Science,0.924275
3,The nanotechnology research has potential appl...,Technology,0.791971
4,The benefits of using digital fashion in fashi...,Fashion,0.865160
...,...,...,...
699,The swimmer broke the national record in the 2...,Sports,0.975228
700,The impact of food deserts on food literacy is...,Food,0.842345
701,The benefits of a healthy lifestyle for reduci...,Health,0.815353
702,The impact of the gig economy on travel experi...,Travel,0.966312


In [5]:
import json

# Loading the existing training data
train_file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Data/train.json'
with open(train_file_path, 'r', encoding='utf-8') as file:
    train_data = json.load(file)

In [6]:
for index, row in df.iterrows():
    category = row['category']
    sentence = row['sentence']
    
    if category in train_data:
        train_data[category].append(sentence)
    else:
        train_data[category] = [sentence]

# Now, train_data is augmented with new sentences


In [7]:
# Save the augmented training data back to a JSON file if needed
augmented_train_file_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Data/augmented_train.json' 
with open(augmented_train_file_path, 'w', encoding='utf-8') as file:
    json.dump(train_data, file, indent=4)


Let's select the remaining sentences that were not yet classified

In [8]:
test_set_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Data/test_shuffle.txt'
with open(test_set_path, 'r', encoding='utf-8') as file:
    test_sentences = [line.strip() for line in file.readlines()]

# Filtering out sentences that are already in the augmented training set
classified_sentences_set = set(df['sentence'].str.lower())
remaining_sentences = [sentence for sentence in test_sentences if sentence.lower() not in classified_sentences_set]


Let's begin the classification of the remaining sentences : 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Flatten the training sentences into a list for vectorization
all_sentences = [sentence for sentences in train_data.values() for sentence in sentences]

# Initialize and fit TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(all_sentences)

# X_train now contains the vectorized form of your augmented training sentences

In [10]:
X_remaining = vectorizer.transform(remaining_sentences)


# KNN :

In [12]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Create numerical labels for your training categories
category_labels = {category: i for i, category in enumerate(train_data.keys())}
# Generate y_train with the correct association of sentences to their category labels
y_train = []
for category, sentences in train_data.items():
    for sentence in sentences:
        y_train.append(category_labels[category])


# Initialize and fit KNN
knn = KNeighborsClassifier(n_neighbors=3)  # Adjust n_neighbors as needed
knn.fit(X_train, y_train)

# Predict categories for the remaining sentences
predicted_labels = knn.predict(X_remaining)

# Convert numerical labels back to category names
predicted_categories = [list(category_labels.keys())[list(category_labels.values()).index(label)] for label in predicted_labels]


In [18]:
new_classifications = pd.DataFrame({
    'sentence': remaining_sentences,
    'category': predicted_categories
})

In [19]:
# Combine the previously classified sentences with the new classifications
combined_classifications = pd.concat([df, pd.DataFrame(new_classifications)], ignore_index=True)



In [20]:
# Check for missing values
print(combined_classifications.isnull().sum())

sentence          0
category          0
probability    1140
dtype: int64


In [21]:
# Create a dictionary for quick lookup
sentence_to_category = {row['sentence'].lower(): row['category'] for index, row in combined_classifications.iterrows()}

# Reorder according to the original test set
ordered_categories = [sentence_to_category[sentence.lower()] for sentence in test_sentences]

In [22]:
# Assuming submission requires a DataFrame with 'Sentence' and 'Category'
submission_df = pd.DataFrame({
    'sentence': test_sentences,
    'category': ordered_categories
})

# Save to CSV for submission
submission_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Bart_MNLI/classification_results_4.csv'
submission_df.to_csv(submission_path, index=False)

In [23]:
# First, rename the columns to match the submission requirements
submission_df.rename(columns={'sentence': 'ID', 'category': 'Label'}, inplace=True)

# Then, replace each sentence with its index
submission_df['ID'] = submission_df.index

# Display the first few rows to verify changes
submission_df.head()


,ID,Label
0,0,Finance
1,1,Environment
2,2,Science
3,3,Technology
4,4,Technology


In [24]:
# Save to CSV for submission
submission_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Bart_MNLI/classification_results_4.csv'
submission_df.to_csv(submission_path, index=False)

# Cosine similarity

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity
similarity_matrix = cosine_similarity(X_remaining, X_train)

In [26]:
# Identify the index of the training sentence with the highest similarity for each remaining sentence
most_similar_indices = np.argmax(similarity_matrix, axis=1)

# Map the indices to categories
training_sentences_categories = [sentence for sentences in train_data.values() for sentence in sentences]
predicted_categories = [training_sentences_categories[index] for index in most_similar_indices]


In [35]:
# Assuming 'most_similar_indices' contains the indices of the most similar training sentences
# First, create a mapping from training sentences to their categories
training_sentences_to_categories = {}
for category, sentences in train_data.items():
    for sentence in sentences:
        # Here we make sure each sentence is mapped to its respective category
        training_sentences_to_categories[sentence] = category

# Now, use the indices to look up the categories for the remaining sentences
predicted_categories = [training_sentences_to_categories[all_sentences[index]] for index in most_similar_indices]


In [36]:
predicted_categories

['Finance',
 'Environment',
 'Science',
 'Technology',
 'Technology',
 'Technology',
 'Travel',
 'Education',
 'Fashion',
 'Entertainment',
 'Health',
 'Environment',
 'Technology',
 'Fashion',
 'Environment',
 'Education',
 'Food',
 'Science',
 'Education',
 'Food',
 'Politics',
 'Environment',
 'Environment',
 'Environment',
 'Health',
 'Health',
 'Environment',
 'Environment',
 'Environment',
 'Environment',
 'Sports',
 'Technology',
 'Politics',
 'Politics',
 'Science',
 'Sports',
 'Fashion',
 'Food',
 'Environment',
 'Politics',
 'Food',
 'Environment',
 'Travel',
 'Environment',
 'Travel',
 'Sports',
 'Travel',
 'Environment',
 'Science',
 'Travel',
 'Food',
 'Education',
 'Travel',
 'Entertainment',
 'Technology',
 'Health',
 'Entertainment',
 'Finance',
 'Sports',
 'Sports',
 'Education',
 'Health',
 'Education',
 'Travel',
 'Fashion',
 'Food',
 'Sports',
 'Science',
 'Sports',
 'Health',
 'Education',
 'Health',
 'Fashion',
 'Health',
 'Health',
 'Fashion',
 'Food',
 'Politics

In [38]:
# Step 1: Create DataFrame for New Classifications
new_classifications_cosine = pd.DataFrame({
    'sentence': remaining_sentences,
    'category': predicted_categories
})

# Step 2: Combine with Previously Classified Sentences
combined_classifications_cosine = pd.concat([df, new_classifications_cosine], ignore_index=True)

# Ensure no missing values in the 'sentence' column
print(combined_classifications_cosine.isnull().sum())



sentence          0
category          0
probability    1140
dtype: int64


In [39]:
# Step 3: Create a dictionary for quick lookup - ensure no missing values or handle them
sentence_to_category_cosine = {row['sentence'].lower(): row['category'] for index, row in combined_classifications_cosine.iterrows()}

# Reorder according to the original test set
ordered_categories_cosine = [sentence_to_category_cosine.get(sentence.lower(), "Unknown") for sentence in test_sentences]

# Prepare the DataFrame for submission
submission_df_cosine = pd.DataFrame({
    'sentence': test_sentences,
    'category': ordered_categories_cosine
})

# Rename columns to 'ID' and 'Label', and replace sentences with indices
submission_df_cosine.rename(columns={'sentence': 'ID', 'category': 'Label'}, inplace=True)
submission_df_cosine['ID'] = submission_df_cosine.index

# Verify the DataFrame structure
print(submission_df_cosine.head())

   ID        Label
0   0      Finance
1   1  Environment
2   2      Science
3   3   Technology
4   4   Technology


In [40]:
# Save to CSV for submission
submission_path = 'C:/Users/Maamar/Desktop/CS_ANLP_KaggComp/Bart_MNLI/cosine_submission.csv'
submission_df_cosine.to_csv(submission_path, index=False)